## 1. Introduction to Asian options and their valuation (arithmetric averaging, fixed strike)

The Asian option is a path-dependent exotic option whose payoff involves a historical average price of the underlying asset.

Due to the averaging mechanism, Asian options have lower volatility and offers greater protection against price fluctuations compared to the plain European counterparts and are prevalent in the commodities, currency and energy markets.

We begin with the geometric Brownian motion (GBM) stock process, $S(t)$ whose dynamics are given by
$$
\begin{equation}
    d S(t)=r S(t) d t+\sigma S(t) d \widetilde{W}(t)
\end{equation}
$$
where $\widetilde{W}(t), 0\leq t\leq T$ denotes the Brownian motion under the risk neutral measure $\widetilde{\mathbb{P}}$.

The payoff of the Asian call with the non-negative fixed-strike $K$ under arithmetic averaging at time $T$ is given by

$$V(T) = \left(\frac{1}{T} \int_0^T S(t) d t-K\right)^{+}$$

Assuming a constant interest rate $r$, the price for $t \leq T$ is given by the risk-neutral pricing formula 

$$ V(t)=\tilde{\mathbb{E}}\left[e^{-r(T-t)} V(T) \mid \mathcal{F}(t)\right], \quad 0 \leq t \leq T$$

Under the Black-Scholes model, the option price is expressed in terms of both $S(t)$ and auxiliary variable $A(t) = \int_0^t S(u) d u$ to obtain the Asian call option PDE:
$$ \frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2 A^2 \frac{\partial^2 V}{\partial A^2} + r A \frac{\partial V}{\partial A} - r V = 0$$

There is no closed form solution formula for price of the Asian option, due to the fact that the arithmetic average is not log-normally distributed.

As such, the option prices often rely on numerical techniques or approximation methods. This will be the focus of this notebook.


## 2. Stock process

### 2.1. Black-Scholes model

The class `GBM` below implements the stock process (geometric Brownian motion) as in (1).

In [1]:
import numpy as np
import scipy
from scipy.stats import norm
import matplotlib.pyplot as plt

class GBM():

    # Initialise GBM model
    def __init__(self, S_0, r, sigma, T, steps):
        self.S_0 = S_0        
        self.r = r          
        self.sigma = sigma  
        self.T = T
        self.steps = steps
        self.dt = T/(steps-1)

    # Generate N paths
    def generate_paths(self, n_paths):

        X_0 = np.zeros((n_paths, 1))
        r, sigma, dt = self.r, self.sigma, self.dt

        # Create normal increments in the exponent
        dW = norm.rvs(loc = (r - sigma**2 / 2) * dt,
                      scale = np.sqrt(dt) * sigma,
                      size = (n_paths, self.steps - 1))

        # Multiply by spot price
        X = np.concatenate((X_0, dW), axis=1).cumsum(1)

        return self.S_0 * np.exp(X)

In [2]:
np.random.seed(0)

S_0 = 100.0  # spot stock price
T = 1  # maturity
r = 0.1  # risk free rate
sigma = 0.2  # diffusion coefficient or volatility

n_steps = 10000 # number of steps

S = GBM(S_0, r, sigma, T, n_steps)

## 3. Option Pricing Methods

### 3.1 Laplace inversion method 

#### Talbot algorithm

In [4]:
from scipy import integrate
from scipy.special import gamma

K = 100 # Strike price
M = 50 # Parameters of the algorithm

S_0, T, r, sigma = S.S_0, S.T, S.r, S.sigma

h = sigma**2 / 4
q = K * T * sigma**2 / (4 * S_0)
nu = 2*r / sigma**2 - 1

d = np.zeros(M, dtype=complex)
d[0] = 2 * M / 5
for k in range(1, M):
    d[k] = 2 * k * np.pi / 5 * (1 / np.tan(k * np.pi / M) + 1j)

g = np.zeros(M, dtype=complex)
g[0] = 0.5 * np.exp(d[0])
for k in range(1, M):
    g[k] = (1 + 1j * (k * np.pi / M) * (1 + (1 / np.tan(k * np.pi / M))**2) - 1j * (1 / np.tan(k * np.pi / M))) * np.exp(d[k])

arg = d / h

if np.any(np.abs(arg) < np.maximum(0, 2 * (nu + 1))):
    raise ValueError('The argument is not in the right half-plane.')

mu = np.sqrt(2 * arg + nu**2)
alpha = (mu + nu) / 2
beta = (mu - nu) / 2

def complex_quadrature(func, a, b):
    def real_func(x): return np.real(func(x))
    def imag_func(x): return np.imag(func(x))

    real_integral = integrate.quad(real_func, a, b)
    imag_integral = integrate.quad(imag_func, a, b)
    return real_integral[0] + 1j*imag_integral[0]

def integrand(u, alpha, beta, q):
    return u**(beta - 2) * (1 - u)**(alpha + 1) * np.exp(-u / (2 * q))

# Handle arrays (integrate over multiple values of alpha, beta)
results = []
for a, b in zip(alpha, beta):
    results.append(complex_quadrature(lambda u: integrand(u, a, b, q), 0.001, 0.999))

results = np.array(results) 

res = (2 * q)**(1 - beta) / (2 * arg * (alpha + 1) * gamma(beta)) * results * g
res = np.real(res)
res =  2 / (5 * h) * np.sum(res)

res *= np.exp(-r * T) * (S_0 * 4) / (T * sigma**2)
res

7.03975000922774